### 랭체인(LangChain) 을 활용하여 허깅페이스(HuggingFace) 허브 
출처 : 테디노트  
랭체인은 허깅페이스 허브의 앤드포인트(Endpoint) 추론을 활용할 수 있는 래퍼(wrapper) 객체 및 함수를 제공하고 있습니다.   우리는 이를 활용하여 보다 쉽게 허깅페이스 모델을 활용한 서비스를 제작할 수 있습니다.  
OpenAI 사의 ChatGPT 비용이 크다면, 공개된 허깅페이스 모델을 활용하는 것도 하나의 대안일 수 있습니다.  

* HuggingFace Hub 소개

Hugging Face Hub는 120k 이상의 모델, 20k의 데이터셋, 그리고 50k의 데모 앱(Spaces)을 포함하는 플랫폼입니다.  
모든 것은 오픈 소스이며 공개적으로 이용할 수 있습니다.  
이 플랫폼에서 사람들은 쉽게 협업하고 함께 ML을 구축할 수 있습니다.  
아래 예시는 Hugging Face Hub에 연결하는 방법과 다양한 모델을 사용하는 방법을 보여줍니다.  

In [ ]:
pip install langchain huggingface_hub transformers datasets dotenv langchain_community transformers 
pip install torch

허깅페이스 토큰 발급  
허깅페이스(https://huggingface.co) 에 회원가입을 한 뒤, 아래의 주소에서 토큰 발급을 신청합니다.  

토큰 발급주소: https://huggingface.co/docs/hub/security-tokens  
토큰을 발급받은 뒤 아래 주소에서 LLM 의 READ 키를 복사합니다.  

LLM 키: https://huggingface.co/settings/tokens  

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('../.env')  # .env 파일의 환경변수 로드
os.environ.get("HUGGINGFACEHUB_API_TOKEN")

!! 추론에 활용할 모델 선택  
허깅페이스 LLM 리더보드: https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard  

리더보드에 게재된 모델의 성능을 직접 확인할 수 있으며, 모델의 ID 만 알고 있으면 됩니다.  
예를 들어, 위의 그림 기준으로 첫 번째 랭크되어 있는 모델의 ID 는 AIDC-ai-business/Marcoroni-70B-v1 입니다.  
한글 LLM 리더보드: https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard  

In [ ]:
# 허깅페이스 0.19.4 + langchain 0.3.25 
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## beomi/KoAlpaca-Polyglot-5.8B모델 사용하기 (메모리기반의 한국어모델)
"""
beomi/KoAlpaca-Polyglot-5.8B는 Hugging Face Inference API로 사용할 수 없습니다.
반드시 로컬에서 transformers로만 사용해야 합니다.
Inference API로 사용하려면, "Use via API"가 있는 모델을 선택하세요.
"""

repo_id = "beomi/KoAlpaca-Polyglot-5.8B"
question = "대한민국 수도는 어디인가요?"
template = """Question: {question}

Answer: """

# 프롬프트 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=["question"])

# HuggingFaceHub 객체 생성
llm = HuggingFaceHub(
    repo_id=repo_id, 
    model_kwargs={"temperature": 0.2, 
                  "max_length": 128}
)

llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question=question)) #### 문제점 추적필요 

### 해당 모델의 경우 API 사용 불가능 

In [ ]:
from transformers import pipeline
## torch 설치 필요 ( 단 python 3.13 에서 torch 설치 불가)

# text-generation 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model="beomi/KoAlpaca-Polyglot-5.8B"
)

# 프롬프트 입력
prompt = "대한민국의 수도는 어디인가요?"

# 텍스트 생성
result = pipe(prompt, max_new_tokens=128, temperature=0.2)

# 결과 출력
print(result[0]['generated_text'])